In [98]:
import pandas as pd
import numpy as np

In [99]:
# Load CSV
file_path = "data/AdaniComparisonData.csv"
df = pd.read_csv(file_path)
df.tail()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades
242,10-Oct-2022,EQ,"3,234.00","3,278.75","3,214.35","3,277.60","3,253.00","3,246.25","3,250.34","3,885.00","1,367.70","15,90,792","5,17,06,07,524.10","64,438"
243,07-Oct-2022,EQ,"3,280.00","3,310.95","3,228.15","3,285.00","3,279.00","3,277.60","3,269.68","3,885.00","1,367.70","19,30,275","6,31,13,83,917.90","65,403"
244,06-Oct-2022,EQ,"3,270.00","3,318.80","3,208.05","3,237.70","3,287.00","3,285.00","3,269.85","3,885.00","1,367.70","28,45,165","9,30,32,69,962.70","1,06,184"
245,04-Oct-2022,EQ,"3,230.00","3,291.20","3,125.00","3,157.30","3,229.00","3,237.70","3,229.86","3,885.00","1,367.70","56,42,467","18,22,43,62,149.00","2,06,194"
246,03-Oct-2022,EQ,"3,440.00","3,460.05","3,110.20","3,455.75","3,164.75","3,157.30","3,239.55","3,885.00","1,367.70","75,78,847","24,55,20,75,642.15","2,42,309"


In [100]:
df.columns = df.columns.str.strip()  # Removes spaces

In [101]:
##### ADD DAY LOGIC HERE FOR WEEK SEPARATION #####
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

# # Add 'Day' column
df['Day'] = df['Date'].dt.day_name()

# Sort by Date (ascending order for correct weekly grouping)
df = df.sort_values(by='Date')

In [102]:
df.head()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,Day
246,2022-10-03,EQ,"3,440.00","3,460.05","3,110.20","3,455.75","3,164.75","3,157.30","3,239.55","3,885.00","1,367.70","75,78,847","24,55,20,75,642.15","2,42,309",Monday
245,2022-10-04,EQ,"3,230.00","3,291.20","3,125.00","3,157.30","3,229.00","3,237.70","3,229.86","3,885.00","1,367.70","56,42,467","18,22,43,62,149.00","2,06,194",Tuesday
244,2022-10-06,EQ,"3,270.00","3,318.80","3,208.05","3,237.70","3,287.00","3,285.00","3,269.85","3,885.00","1,367.70","28,45,165","9,30,32,69,962.70","1,06,184",Thursday
243,2022-10-07,EQ,"3,280.00","3,310.95","3,228.15","3,285.00","3,279.00","3,277.60","3,269.68","3,885.00","1,367.70","19,30,275","6,31,13,83,917.90","65,403",Friday
242,2022-10-10,EQ,"3,234.00","3,278.75","3,214.35","3,277.60","3,253.00","3,246.25","3,250.34","3,885.00","1,367.70","15,90,792","5,17,06,07,524.10","64,438",Monday


In [103]:
# Function to insert empty rows
def insert_empty_rows(df):
    empty_row = pd.DataFrame([[""] * len(df.columns)], columns=df.columns)
    df_with_gaps = pd.concat([pd.concat([df.iloc[[i]], empty_row], ignore_index=True) for i in range(len(df))], ignore_index=True)
    return df_with_gaps

In [107]:
# Group by weeks (Monday-Sunday)
# df['Week'] = df['Date'].dt.to_period('W').astype(str)
df['Week'] = df['Date'].dt.isocalendar().week  # Get ISO week number (Monday-Sunday)
df['Year'] = df['Date'].dt.year  

grouped = [insert_empty_rows(g) for _, g in df.groupby('Week')]

In [110]:
# Combine the weekly separated data
df.head()

,Date,series,OPEN,HIGH,LOW,PREV. CLOSE,ltp,close,vwap,52W H,52W L,VOLUME,VALUE,No of trades,Day,Week,Year
246,2022-10-03,EQ,"3,440.00","3,460.05","3,110.20","3,455.75","3,164.75","3,157.30","3,239.55","3,885.00","1,367.70","75,78,847","24,55,20,75,642.15","2,42,309",Monday,40,2022
245,2022-10-04,EQ,"3,230.00","3,291.20","3,125.00","3,157.30","3,229.00","3,237.70","3,229.86","3,885.00","1,367.70","56,42,467","18,22,43,62,149.00","2,06,194",Tuesday,40,2022
244,2022-10-06,EQ,"3,270.00","3,318.80","3,208.05","3,237.70","3,287.00","3,285.00","3,269.85","3,885.00","1,367.70","28,45,165","9,30,32,69,962.70","1,06,184",Thursday,40,2022
243,2022-10-07,EQ,"3,280.00","3,310.95","3,228.15","3,285.00","3,279.00","3,277.60","3,269.68","3,885.00","1,367.70","19,30,275","6,31,13,83,917.90","65,403",Friday,40,2022
242,2022-10-10,EQ,"3,234.00","3,278.75","3,214.35","3,277.60","3,253.00","3,246.25","3,250.34","3,885.00","1,367.70","15,90,792","5,17,06,07,524.10","64,438",Monday,41,2022


In [111]:
# Save the processed CSV
output_path = "output/step-1-weeks-added.csv"
df.to_csv(output_path, index=False)

print(f"CSV processed and saved as {output_path}.")

CSV processed and saved as output/step-1-weeks-added.csv.
